### Fase 4 Sul Americana - Final

In [1]:
import cartolafc
import pandas as pd
from difflib import get_close_matches
import json
import os

pd.set_option('display.max_columns', 50)            # permite a visualização de 50 colunas do dataframe
pd.options.display.float_format = '{:.2f}'.format   # pandas: para todos os números aparecerem com duas casas decimais

# Cria uma instância da API
api = cartolafc.Api(attempts=5)

2025-08-12 22:57:32,582 - numexpr.utils - INFO - NumExpr defaulting to 8 threads.


### Carregar o arquivo Excel com os classificados e Gerar um Dicionário

In [2]:
CAMINHO_CLASSIFICADOS = "classificados_fase_3.xlsx"
CAMINHO_PERDEDORES = "perdedores_fase_3.xlsx"

# Inicialização segura
nomes_por_id = {}
dados_torneio_semi = []

# Verificar se ambos os arquivos existem
if not os.path.exists(CAMINHO_CLASSIFICADOS) or not os.path.exists(CAMINHO_PERDEDORES):
    print("📁 Aguardando os arquivos de classificados e perdedores serem gerados.")
else:
    try:
        # Carregar arquivos
        df_classificados = pd.read_excel(CAMINHO_CLASSIFICADOS)
        df_perdedores = pd.read_excel(CAMINHO_PERDEDORES)

        # Verificações básicas
        if df_classificados.empty or df_perdedores.empty:
            print("📄 Um dos arquivos existe mas está vazio.")
        else:
            # Filtrar apenas linhas válidas
            df_validos = df_classificados[df_classificados["classificado_id"].notnull()].reset_index(drop=True)
            df_perdedores_validos = df_perdedores[df_perdedores["perdedor_id"].notnull()].reset_index(drop=True)

            if len(df_validos) < 2 or len(df_perdedores_validos) < 2:
                print("⚠️ Aguardando mais confrontos. Ainda não há dados suficientes para montar a final e disputa de 3º lugar.")
            else:
                # Gerar dicionário nome → ID (apenas dos classificados, mas você pode expandir)
                nomes_por_id = dict(zip(df_validos["classificado_id"], df_validos["classificado_nome"]))
                
                # Expandir nomes_por_id com perdedores também
                nomes_perdedores = dict(zip(df_perdedores_validos["perdedor_id"], df_perdedores_validos["perdedor_nome"]))

                display(nomes_por_id, nomes_perdedores)
                print("\n" + "-" * 100 + "\n")

                # Definir confrontos da fase final
                dados_torneio_final = [
                    ("Final", df_validos.loc[0, "classificado_id"]),
                    ("Final", df_validos.loc[1, "classificado_id"]),
                    ("Decisão 3º Lugar", df_perdedores_validos.loc[0, "perdedor_id"]),
                    ("Decisão 3º Lugar", df_perdedores_validos.loc[1, "perdedor_id"]),
                ]

                print("✅ Confrontos da fase final definidos:")
                for linha in dados_torneio_final:
                    print(linha)

    except Exception as e:
        print(f"❌ Erro ao processar os arquivos: {e}")

{24468241: 'Grêmio imortal 37', 25751748: 'MauHumor F.C.'}

{13951133: 'KING LEONN', 44810918: 'lsauer fc'}


----------------------------------------------------------------------------------------------------

✅ Confrontos da fase final definidos:
('Final', 24468241)
('Final', 25751748)
('Decisão 3º Lugar', 13951133)
('Decisão 3º Lugar', 44810918)


In [3]:
# CAMINHO_ARQUIVO = "3_classificados_semi_sula.xlsx"

# # Inicialização segura
# nomes_por_id = {}
# dados_torneio_final = []

# if not os.path.exists(CAMINHO_ARQUIVO):
#     print("📁 Arquivo ainda não existe — aguardando fim da fase anterior.")
# else:
#     try:
#         # Carrega o arquivo
#         df_classificados = pd.read_excel(CAMINHO_ARQUIVO)

#         if df_classificados.empty:
#             print("📄 Arquivo existe, mas ainda está vazio. Rodadas da fase anterior não concluídas.")
#         else:
#             # Filtrar apenas classificados válidos
#             df_validos = df_classificados[df_classificados["classificado_id"].notnull()].reset_index(drop=True)

#             if len(df_validos) < 2:
#                 print(f"⚠️ Apenas {len(df_validos)} classificados encontrados. Aguardando rodada restante.")
#             else:
#                 # Gerar dicionário nome → ID
#                 nomes_por_id = dict(zip(df_validos["classificado_id"], df_validos["classificado_nome"]))
#                 display(nomes_por_id)
#                 print("\n" + "-" * 100 + "\n")

#                 # Confrontos das quartas com ordem 
#                 dados_torneio_final = [
#                     ("Final", df_validos.loc[0, "classificado_id"]),  # V1
#                     ("Final", df_validos.loc[1, "classificado_id"]),  # V2
#                     ("Decisão 3º Lugar", df_validos.loc[2, "classificado_id"]),  # V3
#                     ("Decisão 3º Lugar", df_validos.loc[3, "classificado_id"]),  # V4
#                     # ("Jogo 3 (JG3)", df_validos.loc[4, "classificado_id"]),  # V5
#                     # ("Jogo 3 (JG3)", df_validos.loc[5, "classificado_id"]),  # V6
#                     # ("Jogo 4 (JG4)", df_validos.loc[6, "classificado_id"]),  # V7
#                     # ("Jogo 4 (JG4)", df_validos.loc[7, "classificado_id"]),  # V8                    
#                 ]

#                 print("✅ Confrontos das Quartas de Final:")
#                 for linha in dados_torneio_final:
#                     print(linha)

#     except Exception as e:
#         print(f"❌ Erro ao processar o arquivo: {e}")


In [4]:
# Criar DataFrame base com dados dos confrontos finais
df_dados_torneio = pd.DataFrame(dados_torneio_final, columns=["Jogo", "ID do Time"])

# # Adicionar Nome do Time usando o dicionário de nomes_por_id
# df_dados_torneio["Nome do Time"] = df_dados_torneio["ID do Time"].map(nomes_por_id)

# Unir os dois dicionários antes do .map()
nomes_completos = nomes_por_id.copy()
nomes_completos.update(nomes_perdedores)

df_dados_torneio["Nome do Time"] = df_dados_torneio["ID do Time"].map(nomes_completos)

# Verificação adicional caso o nome não seja encontrado
df_dados_torneio["Nome do Time"] = df_dados_torneio["Nome do Time"].fillna("Desconhecido")

# Adicionar ID no Grupo no formato 1_F, 2_F, 1_D, 2_D etc.
df_dados_torneio["ID no Grupo"] = df_dados_torneio.groupby("Jogo").cumcount() + 1
df_dados_torneio["ID no Grupo"] = df_dados_torneio["ID no Grupo"].astype(str) + "_" + df_dados_torneio["Jogo"].str[0]

# Reorganizar colunas
df_torneio_semi_liberta = df_dados_torneio[["Jogo", "ID do Time", "Nome do Time", "ID no Grupo"]]

# Separar confrontos por jogo
df_liberta_jogo_1 = df_torneio_semi_liberta[df_torneio_semi_liberta["Jogo"] == "Final"]
df_liberta_jogo_2 = df_torneio_semi_liberta[df_torneio_semi_liberta["Jogo"] == "Decisão 3º Lugar"]

# Criar estrutura de grupos
grupos = {
    "Final": df_liberta_jogo_1,
    "Decisão 3º Lugar": df_liberta_jogo_2
}

# Exibir os dois DataFrames
display(df_liberta_jogo_1, df_liberta_jogo_2)

display(df_dados_torneio)

,Jogo,ID do Time,Nome do Time,ID no Grupo
0,Final,24468241,Grêmio imortal 37,1_F
1,Final,25751748,MauHumor F.C.,2_F


,Jogo,ID do Time,Nome do Time,ID no Grupo
2,Decisão 3º Lugar,13951133,KING LEONN,1_D
3,Decisão 3º Lugar,44810918,lsauer fc,2_D


,Jogo,ID do Time,Nome do Time,ID no Grupo
0,Final,24468241,Grêmio imortal 37,1_F
1,Final,25751748,MauHumor F.C.,2_F
2,Decisão 3º Lugar,13951133,KING LEONN,1_D
3,Decisão 3º Lugar,44810918,lsauer fc,2_D


In [5]:
# Rodada 1 - Fase 5 Libertadores (Equivalente a 17º Rodada do Campeonato Brasileiro) 
confrontos_1a_rodada = [
    # Final
    ("Final", "1_F", "2_F"),    

    # Decisão do 3º lugar
    ("Decisão 3º Lugar", "1_D", "2_D")      
]

In [6]:
# Transformar em DataFrame
df_confrontos = pd.DataFrame(confrontos_1a_rodada, columns=["Jogo", "Mandante_ID", "Visitante_ID"])

# Junta com df_torneio para buscar dados dos mandantes
df_mandantes = df_torneio_semi_liberta.rename(columns={
    "ID no Grupo": "Mandante_ID",
    "Nome do Time": "Mandante_Nome",
    "ID do Time": "Mandante_ID_Time"
})[["Jogo", "Mandante_ID", "Mandante_Nome", "Mandante_ID_Time"]]

# Junta com df_torneio para buscar dados dos visitantes
df_visitantes = df_torneio_semi_liberta.rename(columns={
    "ID no Grupo": "Visitante_ID",
    "Nome do Time": "Visitante_Nome",
    "ID do Time": "Visitante_ID_Time"    
})[["Jogo", "Visitante_ID", "Visitante_Nome", "Visitante_ID_Time"]]

display(df_confrontos)

,Jogo,Mandante_ID,Visitante_ID
0,Final,1_F,2_F
1,Decisão 3º Lugar,1_D,2_D


In [7]:
# Transformar em DataFrame
df_confrontos = pd.DataFrame(confrontos_1a_rodada, columns=["Jogo", "Mandante_ID", "Visitante_ID"])
df_confrontos["Rodada"] = 19
df_rodada_15 = df_confrontos.merge(df_mandantes, on=["Jogo", "Mandante_ID"])
df_rodada_15 = df_rodada_15.merge(df_visitantes, on=["Jogo", "Visitante_ID"])

display(df_rodada_15)

,Jogo,Mandante_ID,Visitante_ID,Rodada,Mandante_Nome,Mandante_ID_Time,Visitante_Nome,Visitante_ID_Time
0,Final,1_F,2_F,19,Grêmio imortal 37,24468241,MauHumor F.C.,25751748
1,Decisão 3º Lugar,1_D,2_D,19,KING LEONN,13951133,lsauer fc,44810918


In [8]:
df_rodadas = pd.concat([
    df_rodada_15
], ignore_index=True)

# Ajustar a numeração da rodada para refletir as rodadas do Cartola (Rodada 17)
df_rodadas["Rodada"] = df_rodadas["Rodada"]

df_rodadas.to_excel("4_confrontos_final_sula.xlsx", index=False)

# Exibir os confrontos da fase 5 (Final)
display(df_rodadas.head(8)) 

,Jogo,Mandante_ID,Visitante_ID,Rodada,Mandante_Nome,Mandante_ID_Time,Visitante_Nome,Visitante_ID_Time
0,Final,1_F,2_F,19,Grêmio imortal 37,24468241,MauHumor F.C.,25751748
1,Decisão 3º Lugar,1_D,2_D,19,KING LEONN,13951133,lsauer fc,44810918


In [9]:
# Criar lista de dicionários no formato desejado
confrontos_js_final = []

for _, row in df_rodadas.iterrows():
    confronto = {
        "jogo": row["Jogo"],
        "rodada": int(row["Rodada"]),
        "mandante": {
            "id": int(row["Mandante_ID_Time"]),
            "nome": row["Mandante_Nome"]
        },
        "visitante": {
            "id": int(row["Visitante_ID_Time"]),
            "nome": row["Visitante_Nome"]        }
    }
    confrontos_js_final.append(confronto)

# Converter para JSON formatado
json_str = json.dumps(confrontos_js_final, indent=2, ensure_ascii=False)

# Salvar como arquivo JS com uma variável global
with open("4_confrontos_final_sula.js", "w", encoding="utf-8") as f:
    f.write("const confrontos_final_sula = ")
    f.write(json_str)
    f.write(";")

In [10]:
def exibir_confrontos(df_rodadas, rodada=None, jogo=None):

    colunas = ["Rodada", "Jogo", "Mandante_Nome", "Visitante_Nome"]
    df_filtrado = df_rodadas.copy()

    df_filtrado["Rodada"] = df_filtrado["Rodada"].astype(str) + "ª Rodada"    

    if rodada is not None:
        df_filtrado = df_filtrado[df_filtrado["Rodada"] == rodada]

    if jogo is not None:
        df_filtrado = df_filtrado[df_filtrado["Jogo"] == jogo]

    return df_filtrado[colunas].sort_values(by=["Jogo", "Rodada"])

In [25]:
jogo = "Decisão 3º Lugar"
# jogo = "Final"

# Exibir todos os confrontos do Jogo 1
display(exibir_confrontos(df_rodadas, jogo= f"{jogo}").head(8))

,Rodada,Jogo,Mandante_Nome,Visitante_Nome
1,19ª Rodada,Decisão 3º Lugar,KING LEONN,lsauer fc


In [12]:
def campeonato_comecou(api, ids_times):
    """Verifica se o campeonato já começou observando a pontuação na 1ª rodada."""
    for time_id in ids_times.values():
        try:
            pontuacao = api.time(time_id=time_id, rodada=19).ultima_pontuacao
            if pontuacao is not None:
                return True
        except cartolafc.errors.CartolaFCError:
            continue
    return False

def obter_pontuacao_por_rodada(api, time_id, rodada_atual):
    """Obtém a pontuação do time em cada rodada até a rodada atual."""
    pontuacoes = {}
    for rodada in range(19, rodada_atual):
        try:
            time_rodada = api.time(time_id=time_id, rodada=rodada)
            pontuacoes[rodada] = time_rodada.ultima_pontuacao
        except cartolafc.errors.CartolaFCError as e:
            print(f"Erro ao acessar pontuação da rodada {rodada} para o time {time_id}: {e}")
            pontuacoes[rodada] = None
    return pontuacoes



def gerar_df_pontuacoes(api, ids_times, rodada_inicio=19):
    rodada_atual = api.mercado().rodada_atual

    if not campeonato_comecou(api, ids_times):
        print("📌 A fase ainda não começou. Criando estrutura com placeholders.")
        df = pd.DataFrame(index=ids_times.keys(), columns=[f'Rodada {i}' for i in range(rodada_inicio, rodada_inicio)])
        df[:] = None  # ou 0 se preferir
        return df

    # Campeonato começou
    df = pd.DataFrame()
    for nome, time_id in ids_times.items():
        pontuacoes = obter_pontuacao_por_rodada(api, time_id, rodada_atual)
        df[nome] = pd.Series(pontuacoes)

    df = df.transpose()

    # Ajustar nome das colunas para "Rodada X"
    colunas_numeradas = df.columns.tolist()
    df.columns = [f'Rodada {i}' for i in colunas_numeradas]

    # Adicionar a linha com o time líder por rodada (opcional)
    if not df.empty:
        df.loc['Lider_Rodada'] = df.idxmax()

    return df


In [13]:
# Combina os nomes dos vencedores e perdedores
nomes_completos = nomes_por_id.copy()
nomes_completos.update(nomes_perdedores)

# Agora inverte para gerar os ids_times corretamente
ids_times = {v: k for k, v in nomes_completos.items()}

df_pontuacoes = gerar_df_pontuacoes(api, ids_times)

In [14]:

if df_pontuacoes.empty:
    print("⚠️ Sem pontuações disponíveis ainda.")
else:
    display(df_pontuacoes.T)

,Grêmio imortal 37,MauHumor F.C.,KING LEONN,lsauer fc,Lider_Rodada
Rodada 19,62.95,66.70,64.30,52.00,MauHumor F.C.


In [15]:
def classificacao_por_grupo(df_rodadas, df_pontuacoes):

    df_pontuacoes_times = df_pontuacoes.drop(index='Lider_Rodada', errors='ignore')
    estatisticas = {}

    for _, confronto in df_rodadas.iterrows():
        rodada = confronto["Rodada"]
        mandante = confronto["Mandante_Nome"]
        visitante = confronto["Visitante_Nome"]
        jogo = confronto["Jogo"]
        coluna_rodada = f"Rodada {rodada}"

        if mandante not in df_pontuacoes_times.index or visitante not in df_pontuacoes_times.index:
            continue
        if coluna_rodada not in df_pontuacoes_times.columns:
            continue

        pontos_mandante = df_pontuacoes_times.at[mandante, coluna_rodada]
        pontos_visitante = df_pontuacoes_times.at[visitante, coluna_rodada]

        # Ignorar confrontos ainda não disputados (com pontuação 0 ou ausente)
        if (
            pd.isnull(pontos_mandante) or pd.isnull(pontos_visitante) or
            (pontos_mandante == 0 and pontos_visitante == 0)
        ):
            continue

        for time in [mandante, visitante]:
            if jogo not in estatisticas:
                estatisticas[jogo] = {}
            if time not in estatisticas[jogo]:
                estatisticas[jogo][time] = {
                    "Pontos": 0, "Vitórias": 0, "Empates": 0, "Derrotas": 0,
                    "Total_Cartola": 0,
                    "Cartola_Sofrido": 0
                }

        # Atualizar estatísticas do jogo
        estatisticas[jogo][mandante]["Total_Cartola"] += pontos_mandante
        estatisticas[jogo][mandante]["Cartola_Sofrido"] += pontos_visitante

        estatisticas[jogo][visitante]["Total_Cartola"] += pontos_visitante
        estatisticas[jogo][visitante]["Cartola_Sofrido"] += pontos_mandante

        if pontos_mandante > pontos_visitante:
            estatisticas[jogo][mandante]["Pontos"] += 3
            estatisticas[jogo][mandante]["Vitórias"] += 1
            estatisticas[jogo][visitante]["Derrotas"] += 1
        elif pontos_mandante < pontos_visitante:
            estatisticas[jogo][visitante]["Pontos"] += 3
            estatisticas[jogo][visitante]["Vitórias"] += 1
            estatisticas[jogo][mandante]["Derrotas"] += 1
        else:
            estatisticas[jogo][mandante]["Pontos"] += 1
            estatisticas[jogo][visitante]["Pontos"] += 1
            estatisticas[jogo][mandante]["Empates"] += 1
            estatisticas[jogo][visitante]["Empates"] += 1

    # Gerar DataFrame final
    df_resultado = pd.concat([
        pd.DataFrame({
            "Jogo": jogo,
            "Nome do Time": list(times.keys()),
            "Pontos": [stats["Pontos"] for stats in times.values()],
            "Vitórias": [stats["Vitórias"] for stats in times.values()],
            "Empates": [stats["Empates"] for stats in times.values()],
            "Derrotas": [stats["Derrotas"] for stats in times.values()],
            "Total Cartola": [stats["Total_Cartola"] for stats in times.values()],
            "Cartola Sofrido": [stats["Cartola_Sofrido"] for stats in times.values()],
            "Saldo Cartola": [
                stats["Total_Cartola"] - stats["Cartola_Sofrido"] for stats in times.values()
            ]
        }) for jogo, times in estatisticas.items()
    ], ignore_index=True)

    # Ordenar e adicionar posição
    df_resultado = df_resultado.sort_values(
        by=["Jogo", "Pontos", "Vitórias", "Total Cartola", "Saldo Cartola"],
        ascending=[True, False, False, False, False]
    )

    df_resultado["Posição"] = df_resultado.groupby("Jogo").cumcount() + 1

    df_resultado_por_jogo = {
        jogo: df_resultado[df_resultado["Jogo"] == jogo] for jogo in df_resultado["Jogo"].unique()
    }

    return df_resultado, df_resultado_por_jogo


In [16]:
# Padroniza e exibe as pontuações se existirem
try:
    if 'df_pontuacoes' not in globals() or df_pontuacoes.empty:
        print("📌 Nenhuma pontuação disponível — provavelmente ainda não há classificados definidos.")
    else:
        # Padroniza os nomes no df_rodadas
        df_rodadas["Mandante_Nome"] = df_rodadas["Mandante_Nome"].str.strip()
        df_rodadas["Visitante_Nome"] = df_rodadas["Visitante_Nome"].str.strip()

        # Padroniza os índices do df_pontuacoes
        df_pontuacoes.index = df_pontuacoes.index.str.strip()

        # Exibe
        display(df_pontuacoes)

except NameError as e:
    print("❌ Variável df_rodadas ou df_pontuacoes não está definida ainda.")
except Exception as e:
    print(f"❌ Erro inesperado: {e}")


,Rodada 19
Grêmio imortal 37,62.95
MauHumor F.C.,66.70
KING LEONN,64.30
lsauer fc,52.00
Lider_Rodada,MauHumor F.C.


In [17]:
try:
    # Verifica se as variáveis existem e estão válidas
    if (
        'df_rodadas' not in globals() or df_rodadas.empty or
        'df_pontuacoes' not in globals() or df_pontuacoes.empty
    ):
        print("⚠️ Rodadas ou pontuações ainda não disponíveis — classificação não será gerada.")
    
    elif 'classificacao_por_grupo' not in globals():
        print("⚠️ Função 'classificacao_por_grupo' não está definida.")
    
    else:
        # Gerar a classificação da fase 5
        df_resultado, df_resultado_por_jogo = classificacao_por_grupo(
            df_rodadas, df_pontuacoes
        )

        # Salvar cada grupo em uma aba do Excel
        with pd.ExcelWriter("classificacao_por_grupo_fase_5.xlsx") as writer:
            for jogo, df in df_resultado_por_jogo.items():
                df.to_excel(writer, sheet_name=jogo, index=False)

        # Exibir a classificação geral
        df_resultado_jogo_1 = df_resultado[df_resultado["Jogo"] == "Final"]
        df_resultado_jogo_2 = df_resultado[df_resultado["Jogo"] == "Decisão 3º Lugar"]

        display(df_resultado_jogo_1, df_resultado_jogo_2)

except Exception as e:
    print(f"❌ Erro ao gerar a classificação: {e}")


,Jogo,Nome do Time,Pontos,Vitórias,Empates,Derrotas,Total Cartola,Cartola Sofrido,Saldo Cartola,Posição
1,Final,MauHumor F.C.,3,1,0,0,66.70,62.95,3.75,1
0,Final,Grêmio imortal 37,0,0,0,1,62.95,66.70,-3.75,2


,Jogo,Nome do Time,Pontos,Vitórias,Empates,Derrotas,Total Cartola,Cartola Sofrido,Saldo Cartola,Posição
2,Decisão 3º Lugar,KING LEONN,3,1,0,0,64.30,52.00,12.30,1
3,Decisão 3º Lugar,lsauer fc,0,0,0,1,52.00,64.30,-12.30,2


In [18]:
# Verifica se a variável df_resultado_por_grupo existe e está populada
if 'df_resultado_por_jogo' in locals() and df_resultado_por_jogo:
    # Criar estrutura em formato de dicionário para JSON/JS
    classificacao_final_sula = {}

    for jogo, df in df_resultado_por_jogo.items():
        classificacao_final_sula[jogo] = []

        # Remove duplicatas
        df = df.drop_duplicates(subset=["Nome do Time"])

        for _, row in df.iterrows():
            classificacao_final_sula[jogo].append({
                "posicao": int(row["Posição"]),
                "nome": row["Nome do Time"],
                "pontos": int(row["Pontos"]),
                "vitorias": int(row["Vitórias"]),
                "empates": int(row["Empates"]),
                "derrotas": int(row["Derrotas"]),
                "totalCartola": float(row["Total Cartola"]),
                "cartolaSofrido": float(row["Cartola Sofrido"]),
                "saldoCartola": float(row["Saldo Cartola"])
            })


    # Converter para JSON formatado
    json_str = json.dumps(classificacao_final_sula, indent=2, ensure_ascii=False)

    # Salvar como arquivo JS com uma variável global
    with open("4_classificacao_por_jogo_final_sula.js", "w", encoding="utf-8") as f:
        f.write("const classificacao_final_sula = ")
        f.write(json_str)
        f.write(";")

    print("✅ Arquivo JS salvo com sucesso.")

else:
    print("⚠️ Classificação da Final da Sula indisponíveis. Criando placeholders.")

    # ⚠️ Classificação da fase 4 indisponível — criar estrutura zerada manualmente

    # Dicionário com confrontos definidos
    confrontos_fase_4 = {
        "Final": ["Grêmio imortal 37", "MauHumor F.C."],
        "Decisão 3º Lugar": ["KING LEONN", "lsauer fc"],
        # "Jogo 2 (JG2)": ["MauHumor F.C.", "lsauer fc"],
        # "Jogo 3 (JG3)": ["MauHumor F.C.", "TEAM LOPES 99"],
        # "Jogo 4 (JG4)": ["Tabajara de Inhaua FC2", "lsauer fc"]
    }

    # Estrutura zerada para o JS
    classificacao_final_sula = {}

    for jogo, times in confrontos_fase_4.items():
        classificacao_final_sula[jogo] = []
        for nome_time in times:
            classificacao_final_sula[jogo].append({
                "posicao": 0,
                "nome": nome_time,
                "pontos": 0,
                "vitorias": 0,
                "empates": 0,
                "derrotas": 0,
                "totalCartola": 0.0,
                "cartolaSofrido": 0.0,
                "saldoCartola": 0.0
            })

    # Salvar como JS
    with open("4_classificacao_por_jogo_final_sula.js", "w", encoding="utf-8") as f:
        f.write("const classificacao_final_sula = ")
        json.dump(classificacao_final_sula, f, indent=2, ensure_ascii=False)
        f.write(";")

    print("✅ Arquivo JS com classificação zerada gerado com sucesso.")



✅ Arquivo JS salvo com sucesso.


In [19]:
def exibir_resultados_rodada(df_rodadas, df_pontuacoes, rodada, jogo=None):
    """
    Exibe os resultados de uma rodada específica, com pontuação e dados dos times.
    """

    if rodada not in df_rodadas["Rodada"].values:
        return pd.DataFrame([{
            "Jogo": jogo or "-",
            "Rodada": rodada,
            "Mandante_Nome": "-",          
            "Mandante_Pontos": "-",
            "Visitante_Nome": "-",           
            "Visitante_Pontos": "-",
        }])

    df_filtrado = df_rodadas[df_rodadas["Rodada"] == rodada]
    if jogo:
        df_filtrado = df_filtrado[df_filtrado["Jogo"] == jogo]

    resultados = []

    for _, row in df_filtrado.iterrows():
        jogo_ = row["Jogo"]
        mandante = row["Mandante_Nome"]
        visitante = row["Visitante_Nome"]

        pontos_mandante = df_pontuacoes.get(f"Rodada {rodada}", {}).get(mandante, None)
        pontos_visitante = df_pontuacoes.get(f"Rodada {rodada}", {}).get(visitante, None)

        resultados.append({
            "Jogo": jogo_,
            "Rodada": rodada,
            "Mandante_Nome": mandante,
            "Mandante_Pontos": pontos_mandante,
            "Visitante_Nome": visitante,
            "Visitante_Pontos": pontos_visitante
        })

    return pd.DataFrame(resultados)


In [20]:
# Exibir resultados da 19ª rodada
df_resultados_rodada_19 = exibir_resultados_rodada(df_rodadas, df_pontuacoes, rodada=19)

# Exibir apenas os resultados do df_resultados_jogo_1
df_resultados_jogo_1 = exibir_resultados_rodada(df_rodadas, df_pontuacoes, rodada=19, jogo="Final")

# Exibir
display(df_resultados_rodada_19)

,Jogo,Rodada,Mandante_Nome,Mandante_Pontos,Visitante_Nome,Visitante_Pontos
0,Final,19,Grêmio imortal 37,62.95,MauHumor F.C.,66.70
1,Decisão 3º Lugar,19,KING LEONN,64.30,lsauer fc,52.00


In [21]:
# Criar arquivo com uma aba para cada rodada contendo os resultados detalhados
from pathlib import Path

# Caminho do arquivo de saída
caminho_resultados = "4_resultados_final_sula.xlsx"

# Descobrir as rodadas únicas no DataFrame
rodadas_disponiveis = sorted(df_rodadas["Rodada"].unique())

with pd.ExcelWriter(caminho_resultados) as writer:
    for rodada in rodadas_disponiveis:
        df_resultados = exibir_resultados_rodada(df_rodadas, df_pontuacoes, rodada=rodada)
        nome_aba = f"Rodada {rodada}"
        df_resultados.to_excel(writer, sheet_name=nome_aba, index=False)

print(f"Arquivo salvo com sucesso: {Path(caminho_resultados).resolve()}")

display(df_resultados)

Arquivo salvo com sucesso: C:\Users\ferna\Projetos\GitHub\cartola_fbc\sulamericana\datasets_sula\4_resultados_final_sula.xlsx


,Jogo,Rodada,Mandante_Nome,Mandante_Pontos,Visitante_Nome,Visitante_Pontos
0,Final,19,Grêmio imortal 37,62.95,MauHumor F.C.,66.70
1,Decisão 3º Lugar,19,KING LEONN,64.30,lsauer fc,52.00


In [22]:
resultados_js = []

for rodada in sorted(df_rodadas["Rodada"].unique()):
    df_resultados = exibir_resultados_rodada(df_rodadas, df_pontuacoes, rodada=rodada)
    
    for _, row in df_resultados.iterrows():
        resultado = {
            "jogo": row["Jogo"],
            "rodada": int(rodada),
            "mandante": {
                "nome": row["Mandante_Nome"],
                "pontos": float(row["Mandante_Pontos"]) if row["Mandante_Pontos"] is not None else None
            },
            "visitante": {
                "nome": row["Visitante_Nome"],
                "pontos": float(row["Visitante_Pontos"]) if row["Visitante_Pontos"] is not None else None               
            },
            "vencedor": (
                "mandante" if row["Mandante_Pontos"] is not None and row["Visitante_Pontos"] is not None and row["Mandante_Pontos"] > row["Visitante_Pontos"]
                else "visitante" if row["Mandante_Pontos"] is not None and row["Visitante_Pontos"] is not None and row["Mandante_Pontos"] < row["Visitante_Pontos"]
                else "empate" if row["Mandante_Pontos"] == row["Visitante_Pontos"] and row["Mandante_Pontos"] is not None
                else "indefinido"
            )

        }
        resultados_js.append(resultado)


# Exportar para arquivo .js
import json

with open("4_resultados_final_sula.js", "w", encoding="utf-8") as f:
    f.write("const resultados_final_sula = ")
    f.write(json.dumps(resultados_js, indent=2, ensure_ascii=False))
    f.write(";")


In [23]:
def obter_classificados_com_id(resultados, ids_por_nome):
    classificados = []
    jogos_agrupados = {}
    for jogo in resultados:
        chave = jogo['jogo']
        if chave not in jogos_agrupados:
            jogos_agrupados[chave] = []
        jogos_agrupados[chave].append(jogo)

    for chave, partidas in jogos_agrupados.items():
        if len(partidas) < 2:
            continue

        partidas = sorted(partidas, key=lambda x: x['rodada'])
        ida, volta = partidas

        if ida['mandante']['pontos'] is None or volta['mandante']['pontos'] is None:
            continue

        time1 = ida['mandante']['nome']
        time2 = ida['visitante']['nome']

        pontos_time1 = ida['mandante']['pontos'] + volta['visitante']['pontos']
        pontos_time2 = ida['visitante']['pontos'] + volta['mandante']['pontos']

        if pontos_time1 > pontos_time2:
            vencedor = time1
        elif pontos_time2 > pontos_time1:
            vencedor = time2
        else:
            vencedor = "EMPATE"

        classificados.append({
            "jogo": chave,
            "classificado_nome": vencedor,
            "classificado_id": ids_por_nome.get(vencedor) if vencedor in ids_por_nome else None
        })

    return classificados


In [24]:
# Inverter os nomes
ids_por_nome = {v: k for k, v in nomes_por_id.items()}

# Ler o resultados_final_sula.js (como já fizemos antes)
with open("4_resultados_final_sula.js", "r", encoding="utf-8") as f:
    conteudo = f.read()

conteudo_json = conteudo.replace("const resultados_final_sula = ", "").strip().rstrip(";")
resultados = json.loads(conteudo_json)

# Obter os classificados
classificados = obter_classificados_com_id(resultados, ids_por_nome)

# Salvar
df_classificados = pd.DataFrame(classificados)
df_classificados.to_excel("4_classificados_final_sula.xlsx", index=False)

with open("4_classificados_final_sula.js", "w", encoding="utf-8") as f:
    f.write("const classificados_semi_sula = ")
    json.dump(classificados, f, ensure_ascii=False, indent=2)
    f.write(";")
print("✅ Classificados salvos com sucesso em '4_classificados_final_sula.xlsx' e '4_classificados_final_sula.js'.")

# Exibir para validar
print("Classificados encontrados:", classificados)


✅ Classificados salvos com sucesso em '4_classificados_final_sula.xlsx' e '4_classificados_final_sula.js'.
Classificados encontrados: []


### Definição dos Confrontos das Rodadas da Fase 4 da Sul Americana (Rodada 19)